In [6]:
import sys
from pathlib import Path

In [7]:
src_path = Path('../src').resolve()
sys.path.append(str(src_path))

In [8]:
from sqlmodel import Session, select
from api.events.models import EventModel
from api.db.session import engine

In [9]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)

SELECT eventmodel.id, eventmodel.time, eventmodel.updated_at, eventmodel.page, eventmodel.description 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10


In [34]:
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint
from sqlalchemy import func
from datetime import datetime, timedelta, timezone


bucket = time_bucket('1 hour', EventModel.time)
pages = ['/contact', '/about', '/home', '/pricing', '/blog']

start_time = datetime.now(timezone.utc) - timedelta(hours=1)
finish_time = datetime.now(timezone.utc) + timedelta(hours=1)


with Session(engine) as session:
    query = select(
        bucket,
        EventModel.page,
        func.count()
    ).where(
        EventModel.time > start_time,
        EventModel.time <= finish_time,
        EventModel.page.in_(pages)
    ).group_by(
        bucket,
        EventModel.page
    ).order_by(
        bucket,
        EventModel.page
    )
    compiled_query = query.compile(compile_kwargs={'literal_binds': True})
    print(compiled_query)
    print('--------------------------------')
    results = session.exec(query).fetchall()
    pprint(results)

SELECT time_bucket('1 hour'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time > '2025-04-28 13:54:22.091042+00:00' AND eventmodel.time <= '2025-04-28 15:54:22.091616+00:00' AND eventmodel.page IN ('/contact', '/about', '/home', '/pricing', '/blog') GROUP BY time_bucket('1 hour'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 hour'::interval, eventmodel.time), eventmodel.page
--------------------------------
[(datetime.datetime(2025, 4, 28, 14, 0, tzinfo=datetime.timezone.utc), '/about', 2115),
 (datetime.datetime(2025, 4, 28, 14, 0, tzinfo=datetime.timezone.utc), '/blog', 1909),
 (datetime.datetime(2025, 4, 28, 14, 0, tzinfo=datetime.timezone.utc), '/contact', 1984),
 (datetime.datetime(2025, 4, 28, 14, 0, tzinfo=datetime.timezone.utc), '/home', 2033),
 (datetime.datetime(2025, 4, 28, 14, 0, tzinfo=datetime.timezone.utc), '/pricing', 1959)]
